In [1]:
import catboost as cb
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn as skl
from sklearn.metrics import confusion_matrix, accuracy_score
import skopt
import scipy
from skopt.space import Real

PREDICCION_REAL = False
MAXIMIZAR_HIPERPARAMETROS = False
PARAMETROS = {
    "iterations" : 43,
    'learning_rate': 0.02421554619906638,
    "random_seed" : 32574,
    "l2_leaf_reg" : 0.0025186975910250903,
    'subsample': 0.922332290245647,
    "random_strength" : 0.2856618944034107,
    'depth': 2,
    "rsm" : 0.9371296456479311,
    "early_stopping_rounds" : 15,
    "border_count" : 1297,
    "loss_function":'Logloss',
    "verbose":True,
    "has_time" : True
}

In [2]:
#APERTURA DE ARCHIVO DE ARCHIVOS
entrenamiento = pd.read_csv("entrenamiento-listo.csv")
test = pd.read_csv("test-listo.csv")

In [3]:
#FILTRADO DE COLUMNAS - NO REMOVER STAGE O FECHA
fugas = ['ID','Opportunity_Name','Sales_Contract_No','Account_Name','Account_Owner','Opportunity_Owner','Last_Modified_By','Product_Family','Product_Name','ASP','ASP_(converted)']
#'Total_Taxable_Amount'
columnas_fecha = ['Month','Last_Modified_Date','Account_Created_Date','Opportunity_Created_Date','Quote_Expiry_Date','Planned_Delivery_Start_Date','Planned_Delivery_End_Date']
#entrenamiento = entrenamiento.drop(columns=fugas)
#test = test.drop(columns=fugas)
#test = test.drop(columns=columnas_fecha)
#entrenamiento = entrenamiento.drop(columns=columnas_fecha)

In [4]:
entrenamiento.sort_values("Fecha")
objetivo = (entrenamiento['Stage'] == 'Closed Won').astype(int)
entrenamiento = entrenamiento.drop(columns=['Stage',"Fecha"])
columnas_object = list(entrenamiento.select_dtypes(include=['object']).columns)
if 'Stage' in columnas_object : columnas_object.remove('Stage')

In [5]:
if not PREDICCION_REAL:
    test_label = (test['Stage'] == 'Closed Won').astype(int)
    test = test.drop(columns=['Stage'])

In [6]:
entrenamiento_pool = cb.Pool(entrenamiento, objetivo,cat_features=columnas_object)
test_pool = cb.Pool(test, cat_features=columnas_object)

In [7]:
model = cb.CatBoostClassifier(**PARAMETROS)

model.fit(entrenamiento_pool)
# make the prediction using the resulting model
preds = model.predict(test_pool)
print("class = ", preds)

0:	learn: 0.6476018	total: 187ms	remaining: 7.86s
1:	learn: 0.6056161	total: 210ms	remaining: 4.31s
2:	learn: 0.5663749	total: 242ms	remaining: 3.23s
3:	learn: 0.5306039	total: 258ms	remaining: 2.52s
4:	learn: 0.4981081	total: 274ms	remaining: 2.08s
5:	learn: 0.4911753	total: 284ms	remaining: 1.75s
6:	learn: 0.4618661	total: 322ms	remaining: 1.65s
7:	learn: 0.4357998	total: 332ms	remaining: 1.45s
8:	learn: 0.4124166	total: 369ms	remaining: 1.39s
9:	learn: 0.3891262	total: 385ms	remaining: 1.27s
10:	learn: 0.3692699	total: 415ms	remaining: 1.21s
11:	learn: 0.3501674	total: 431ms	remaining: 1.11s
12:	learn: 0.3322585	total: 460ms	remaining: 1.06s
13:	learn: 0.3160195	total: 471ms	remaining: 976ms
14:	learn: 0.3000913	total: 509ms	remaining: 951ms
15:	learn: 0.2852298	total: 525ms	remaining: 887ms
16:	learn: 0.2730988	total: 557ms	remaining: 851ms
17:	learn: 0.2607626	total: 572ms	remaining: 794ms
18:	learn: 0.2492240	total: 586ms	remaining: 740ms
19:	learn: 0.2396810	total: 601ms	remaini

In [8]:
resultados = test[['Opportunity_ID']].copy()
resultados['Target'] = pd.Series(preds)
resultados = resultados.groupby('Opportunity_ID').mean()
resultados = resultados.reset_index()
#resultados['Target'] = resultados['Target'].apply(lambda x: int(x >= 0.5))    
resultados.to_csv("prediccion.csv", index=False)
resultados['Target'].value_counts()

1    1548
0    1014
Name: Target, dtype: int64

In [9]:
if not PREDICCION_REAL:
    cm = confusion_matrix(test_label, preds)
    print(cm)
    accuracy_score(test_label, preds)

[[1511  204]
 [  28 2687]]


In [10]:
accuracy_score(test_label, preds)

0.9476297968397291

In [11]:
model.get_feature_importance(prettified=True)

,Feature Id,Importances
0,Sales_Contract_No,95.872901
1,Last_Modified_Date,1.292534
2,Planned_Delivery_Start_Date,1.211155
3,Account_Name,0.583702
4,Quote_Expiry_Date,0.323244
5,Last_Modified_By,0.304985
6,Total_Amount_USD,0.190890
7,Month,0.156477
8,Opportunity_Duration,0.064111
9,ID,0.000000


In [12]:
preds = model.predict(entrenamiento_pool)
accuracy_score(objetivo, preds)

0.9916885389326334

In [13]:
if(not MAXIMIZAR_HIPERPARAMETROS): sys.exit()

NameError: name 'sys' is not defined

In [ ]:
import category_encoders
def categoricas_a_numericas(train,test,label,usar_label):
    if (usar_label):
        columnas_object = list(train.select_dtypes(include=['object']).columns)
    else:
        columnas_object = list(test.select_dtypes(include=['object']).columns)
    if 'Stage' in columnas_object : columnas_object.remove('Stage')
    ohe = category_encoders.cat_boost.CatBoostEncoder(cols = columnas_object,return_df = True)
    ohe.fit(train,label)
    if (usar_label):
        columnas = ohe.transform(train,label)
        for columna in columnas_object:
            train[columna] = columnas[columna].copy()
    else:
        columnas = ohe.transform(test)
        for columna in columnas_object:
            test[columna] = columnas[columna].copy()
categoricas_a_numericas(entrenamiento,test,objetivo,False)
categoricas_a_numericas(entrenamiento,test,objetivo,True)
def convertir_a_int(x):
    columnas_float = list(x.select_dtypes(include=[np.float]).columns)
    print(columnas_float)
    for columna in columnas_float:
        x.replace([np.inf, -np.inf], np.nan,inplace=True)
        copia = x[[columna]].copy().dropna(how="all")
        x[columna] = (copia[columna]*100).astype(int)
convertir_a_int(entrenamiento)
convertir_a_int(test)

In [ ]:
search_space = {
        "iterations" : (5, 100),
        'learning_rate': Real(low=0.01, high=1, prior='log-uniform'),
        "random_seed" : (1,40000),
        "l2_leaf_reg" : Real(low=1e-9, high=1000, prior='log-uniform'),
        'subsample': Real(low=0.01, high=1, prior='uniform'),
        "random_strength" : Real(low=1e-9, high=1000, prior='log-uniform'),
        'depth': (1, 16),
        "rsm" : Real(low=0.01, high=1, prior='uniform'),
        "early_stopping_rounds" : (1, 20),
        "border_count" : (1,65535)
    }

search = skopt.BayesSearchCV(estimator=cb.CatBoostClassifier(loss_function='Logloss',has_time=True), search_spaces=search_space, n_jobs=1,n_iter = 10,cv=skl.model_selection.TimeSeriesSplit())
search.fit(entrenamiento.values,objetivo.values)
# report the best result
print(search.best_score_)
print(search.best_params_)

['Region', 'Territory', 'Bureaucratic_Code', 'Source ', 'Billing_Country', 'Account_Name', 'Opportunity_Name', 'Sales_Contract_No', 'Account_Owner', 'Opportunity_Owner', 'Account_Type', 'Opportunity_Type', 'Delivery_Terms', 'Quote_Expiry_Date', 'Last_Modified_By', 'Product_Family', 'Product_Name', 'ASP_Currency', 'ASP', 'ASP_(converted)', 'Planned_Delivery_End_Date', 'Delivery_Quarter', 'Total_Amount_Currency', 'Total_Taxable_Amount_Currency', 'Total_Amount_USD', 'Total_Taxable_Amount_USD', 'Total_Amount_sobre_Total_Taxable_Amount', 'Duracion_Familia', 'Duracion_Territory']
['Region', 'Territory', 'Bureaucratic_Code', 'Source ', 'Billing_Country', 'Account_Name', 'Opportunity_Name', 'Sales_Contract_No', 'Account_Owner', 'Opportunity_Owner', 'Account_Type', 'Opportunity_Type', 'Delivery_Terms', 'Quote_Expiry_Date', 'Last_Modified_By', 'Product_Family', 'Product_Name', 'ASP_Currency', 'ASP', 'ASP_(converted)', 'Planned_Delivery_End_Date', 'Delivery_Quarter', 'Total_Amount_Currency', 'Tot

NameError: name 'search' is not defined

0:	learn: 0.6487288	total: 11.8ms	remaining: 494ms
1:	learn: 0.6076207	total: 38.6ms	remaining: 792ms
2:	learn: 0.5697954	total: 69.1ms	remaining: 921ms
3:	learn: 0.5348484	total: 80.6ms	remaining: 786ms
4:	learn: 0.4892818	total: 119ms	remaining: 902ms
5:	learn: 0.4493662	total: 130ms	remaining: 800ms
6:	learn: 0.4232562	total: 165ms	remaining: 846ms
7:	learn: 0.3995671	total: 179ms	remaining: 784ms
8:	learn: 0.3692861	total: 213ms	remaining: 803ms
9:	learn: 0.3492749	total: 243ms	remaining: 801ms
10:	learn: 0.3325144	total: 274ms	remaining: 799ms
11:	learn: 0.3153793	total: 291ms	remaining: 752ms
12:	learn: 0.2961218	total: 323ms	remaining: 746ms
13:	learn: 0.2820707	total: 338ms	remaining: 699ms
14:	learn: 0.2693704	total: 370ms	remaining: 691ms
15:	learn: 0.2525929	total: 381ms	remaining: 643ms
16:	learn: 0.2412854	total: 418ms	remaining: 640ms
17:	learn: 0.2267741	total: 434ms	remaining: 602ms
18:	learn: 0.2141277	total: 450ms	remaining: 568ms
19:	learn: 0.2066345	total: 460ms	rem

0.9476297968397291

,Feature Id,Importances
0,Sales_Contract_No,66.982593
1,Opportunity_Name,30.835772
2,Planned_Delivery_Start_Date,1.122704
3,Last_Modified_Date,0.928670
4,Quote_Expiry_Date,0.130262
5,ID,0.000000
6,Region,0.000000
7,Territory,0.000000
8,"Pricing, Delivery_Terms_Quote_Appr",0.000000
9,"Pricing, Delivery_Terms_Approved",0.000000
